In [ ]:
"""
Adapted from Modeling and Simulation in Python, Allen Downey
"""

from pandas import read_html
filename = 'Estimates.html'
tables = read_html(filename, header=0, index_col=0, decimal='M')
table2 = tables[2]
''' Exploring the data  
    ------------------
'''
print("head is", table2.head())
print("shape is", table2.shape)
# Out[17]: (67, 24)
print("Columns are", table2.columns)
"""
The column labels are long strings, which makes them hard to work with. 
Let's replace them with shorter strings
"""
table2.columns = ['census', 'prb', 'un', 'maddison', 'hyde',  
                  'biraben', 'mj', 'thomlison', 'durand', 'clark']
print("Revised columns are", table2.columns)

# Here are the estimates from US census bureau
# 1e9 is a shorter way to write 1000000000 or one billion
census = table2.census / 1e9
# result is a pandas series similar to timeseries
print("census.tail() is\n", census.tail())

# Here are the estimates from UN dept of economic and social affairs
un = table2.un / 1e9
print("un tail is\n", un.tail())

def plot_estimates():
    census.plot(style=':', label='US census', xlabel='Year', ylabel='World population (billions)', legend="1" )
    un.plot(style='--', label='DESA', legend="1", title="Data modeling")

In [ ]:
plot_estimates()

In [ ]:
''' Absolute and relative errors  
    ----------------------------
Estimates of world population from the two sources differ slightly. 
One way to characterize this
difference is absolute error, which is the absolute 
value of the difference between the estimates.
To compute absolute errors, we can import abs from NumPy
    
'''
from numpy import abs
abs_error = abs(un - census)
print("absolute error tail ", abs_error.tail())
# when you subtract two serier objects the result is a new series
from numpy import mean
print("mean abs_error", mean(abs_error))
# on average estimates diff by 0.029 billion
from numpy import max
print("max absolute error is", max(abs_error))
# In the worst case, they differ by ~0.1 billion. That is a lot of people
# another way to quantify the difference is relative error, which is error
# divided by the estimates themselves
rel_error = 100*abs_error / census
print("relative error tail", rel_error.tail())
# Now we can interpret the results as a percentage. In 2015 the difference b/w
# the estimates is ~1.4 percent which also happens to be the max
print("mean relative error", mean(rel_error))
# mean relative error is about 0.6 percent
print("max relative error", max(rel_error))


In [ ]:
''' Modeling population growth
    --------------------------
'''
# total_growth during 1950 and 2016 is
total_growth = census[2016] - census[1950]
t_0 = census.index[0]
t_end = census.index[-1]

elapsed_time = t_end - t_0
p_0 = census[t_0]
p_end = census[t_end]

total_growth = p_end - p_0
annual_growth = total_growth/ elapsed_time
print("annual growth", annual_growth)


In [ ]:
''' simulating population growth
    ----------------------------
'''
import pandas as pd
def TimeSeries(*args, **kwargs):
    """Make a pd.Series object to represent a time series.
    """
    if args or kwargs:
        #underride(kwargs, dtype=float)
        series = pd.Series(*args, **kwargs)
    else:
        series = pd.Series([], dtype=float)

    series.index.name = 'Time'
    if 'name' not in kwargs:
        series.name = 'Quantity'
    return series

results = TimeSeries()
results[t_0] = p_0
# Now set the rest of the values by simulating annual growth
# assuming constant growth
for t in range(t_0, t_end):
    results[t+1] = results[t] + annual_growth
results.plot(label='model', title='Constant growth model', legend="1", xlabel='Year', ylabel='World population (billions)')
# from 1950 to 1990, the model does not fit the data particularly well,
# but after that it fits ok
plot_estimates()

In [ ]:
# system object is a dictionary that stores the state information

system = dict()
system['t_0'] = t_0
system['t_end'] = t_end
system['p_0'] = p_0
system['annual_growth'] = annual_growth

print(system)

''' Proportional growth model
    -------------------------
    growth is proportional to population
    net_growth = parameter_alpha * pop
'''

system['birth_rate'] = 25 / 1000 # such that it fits the model
# based on 2020 global rate indexmundi.com/world/death_rate.html
system['death_rate'] = 7.7 / 1000

def run_simulation(system):
    results = TimeSeries()
    results[system['t_0']] = system['p_0']
    
    for t in range(system['t_0'], system['t_end']):
        #results[t+1] = results[t] + system['annual_growth'] # constant growth
        births = system['birth_rate'] * results[t]
        deaths = system['death_rate'] * results[t]
        results[t+1] = results[t] + births - deaths
        
    return results

results_prop = run_simulation(system)
results_prop.plot(label='prop model', legend="1", xlabel='Year', ylabel='World population (billions)')
# proportional model fits the data well from 1950 to 1965 but
# not so well after that
plot_estimates()

In [ ]:
''' Quadratic growth
    ----------------
    net_growth = alpha*pop + beta * pop**2
'''
# these params were chosen by the author by trial and error
system['alpha'] = 25 / 1000
system['beta'] = -1.8 / 1000

def run_simulation_g(system, growth_func):
    results = TimeSeries()
    results[system['t_0']] = system['p_0']
    
    for t in range(system['t_0'], system['t_end']):
        growth = growth_func(t, results[t], system)
        results[t+1] = results[t] + growth
        
    return results

def growth_func_quad(t, pop, system):
    return system['alpha']*pop + system['beta']*pop*pop

results_quad = run_simulation_g(system, growth_func_quad)
results_quad.plot(label='quad model', legend="1", color="blue")
# The model fits the data well over the whole range with just a bit of space between them in the 1960s
plot_estimates()

In [ ]:
'''Generating Projections
   ----------------------
'''
system['t_end'] = 2100
results_far = run_simulation_g(system, growth_func_quad)
results_far.plot(label='proj', legend="1")
print("projection tail", results_far.tail())
plot_estimates()